In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU not found. Training will run on CPU.")


CUDA Available: True
GPU: NVIDIA RTX A4000


In [2]:
pip install lightly


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/855.8 kB ? eta -:--:--
   ---------------------------------------- 855.8/855.8 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/825.4 kB ? eta -:--:--
   ---------------------------------------- 825.4/825.4 kB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/963.5 kB ? eta -:--:--
   ---------------------------------------- 963.5/963.5 kB 9.0 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144614 sha256=3e7dc80607455e389633f5fbcc2c62a883e7641a407fe654a28e03038788beb7
  Stored in directory: c:\users\student\appdata\local\pip\cache\wheels\12\93\dd\1f6a127edc45659556564c5730f6d4e30088

  DEPRECATION: Building 'antlr4-python3-runtime' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'antlr4-python3-runtime'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [3]:
import torch
import torch.nn as nn
from torchvision.models import resnet18
from lightly.models import SimCLR

# === Step 1: Check GPU
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Step 2: Build ResNet backbone without final classification layer
resnet = resnet18(pretrained=False)
backbone = nn.Sequential(*list(resnet.children())[:-1])  # Remove final FC layer

# === Step 3: Load SimCLR model
model = SimCLR(backbone, num_ftrs=512, out_dim=128)
model.to(device)

print("SimCLR model loaded and moved to", device)


CUDA Available: True
GPU: NVIDIA RTX A4000


c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\lightly\models\simclr.py:39: Warning: The high-level building block SimCLR will be deprecated in version 1.3.0. Use low-level building blocks instead. See https://docs.lightly.ai/self-supervised-learning/lightly.models.html for more information
  warnings.warn(


SimCLR model loaded and moved to cuda


In [4]:
pip install scikit-learn



   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/10.7 MB 10.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/10.7 MB 6.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/10.7 MB 4.8 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/10.7 MB 4.1 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/10.7 MB 3.8 MB/s eta 0:00:02
   ----------------- ---------------------- 4.7/10.7 MB 3.6 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.7 MB 3.5 MB/s eta 0:00:02
   --------------------- ------------------ 5.8/10.7 MB 3.3 MB/s eta 0:00:02
   ---------------------- ----------------- 6.0/10.7 MB 3.2 MB/s eta 0:00:02
   ------------------------ --------------- 6.6/10.7 MB 3.1 MB/s eta 0:00:02
   -------------------------- ------------- 7.1/10.7 MB 3.0 MB/s eta 0:00:02
   ---------------------------- ----------- 7.6/10.7 MB 3.0 MB/s eta 0:00:02
   -

In [7]:
import os
import shutil
import random
import uuid
from tqdm import tqdm

# ------------------------------------------------------------------
# 1. Paths
# ------------------------------------------------------------------
SRC_ROOT  = r'E:\dish dataset\Fish Data'       # original root with class folders
DEST_ROOT = r'E:\dish dataset\Fish Data Split' # new flat split

# ------------------------------------------------------------------
# 2. Split ratios
# ------------------------------------------------------------------
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.70, 0.15, 0.15   # must sum to 1

# ------------------------------------------------------------------
# 3. Make destination folders
# ------------------------------------------------------------------
for split in ('train', 'val', 'test'):
    os.makedirs(os.path.join(DEST_ROOT, split), exist_ok=True)

# ------------------------------------------------------------------
# 4. Collect *all* image paths, ignoring sub‑folder (class) names
# ------------------------------------------------------------------
all_images = []
for class_name in os.listdir(SRC_ROOT):
    class_path = os.path.join(SRC_ROOT, class_name)
    if not os.path.isdir(class_path):
        continue

    for root, _, files in os.walk(class_path):
        for f in files:
            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                all_images.append(os.path.join(root, f))

random.shuffle(all_images)

# ------------------------------------------------------------------
# 5. Determine split indices
# ------------------------------------------------------------------
n_total   = len(all_images)
n_train   = int(TRAIN_RATIO * n_total)
n_val     = int(VAL_RATIO   * n_total)
# everything else → test
train_imgs = all_images[:n_train]
val_imgs   = all_images[n_train:n_train + n_val]
test_imgs  = all_images[n_train + n_val:]

# ------------------------------------------------------------------
# 6. Helper to copy while avoiding name collisions
# ------------------------------------------------------------------
def flat_copy(src_paths, split_name):
    dest_folder = os.path.join(DEST_ROOT, split_name)
    for src in tqdm(src_paths, desc=f'Copying {split_name}', leave=False):
        # If different classes share filenames, make them unique
        basename = os.path.basename(src)
        dest     = os.path.join(dest_folder, basename)

        # If the name already exists, append a unique suffix
        if os.path.exists(dest):
            stem, ext = os.path.splitext(basename)
            dest = os.path.join(dest_folder, f'{stem}_{uuid.uuid4().hex[:8]}{ext}')

        shutil.copy(src, dest)

# ------------------------------------------------------------------
# 7. Perform the copies
# ------------------------------------------------------------------
flat_copy(train_imgs, 'train')
flat_copy(val_imgs,   'val')
flat_copy(test_imgs,  'test')

print(f'✅ Done! Total images: {n_total} '
      f'(train {len(train_imgs)}, val {len(val_imgs)}, test {len(test_imgs)})')


✅ Done! Total images: 26950 (train 18865, val 4042, test 4043)


In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from lightly.data import LightlyDataset
from lightly.data.collate import SimCLRCollateFunction
from lightly.models import SimCLR
from lightly.loss import NTXentLoss
from torchvision.models import resnet18
import os

# Paths
dataset_path = r'E:\dish dataset\Fish Data Split\train'
checkpoint_dir = r'E:\dish dataset\checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Dataset and DataLoader
dataset = LightlyDataset(input_dir=dataset_path)
collate_fn = SimCLRCollateFunction(input_size=224)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn, num_workers=4)

# Model: ResNet18 backbone without final FC
resnet = resnet18(pretrained=False)
backbone = nn.Sequential(*list(resnet.children())[:-1])  # remove last fc layer
model = SimCLR(backbone, num_ftrs=512, out_dim=128)

# Loss and optimizer
criterion = NTXentLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for (x0, x1), _, _ in dataloader:
        x0, x1 = x0.to(device), x1.to(device)
        z0, z1 = model(x0), model(x1)
        loss = criterion(z0, z1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    # Save checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"simclr_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Checkpoint saved to {checkpoint_path}")

print("✅ Pretraining Complete.")


c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Using device: cuda
Epoch 1/10 - Loss: 3.6113
Epoch 2/10 - Loss: 3.1203
Epoch 3/10 - Loss: 2.9627
Epoch 4/10 - Loss: 2.8771
Epoch 5/10 - Loss: 2.8238
Checkpoint saved to E:\dish dataset\checkpoints\simclr_epoch_5.pth
Epoch 6/10 - Loss: 2.7868
Epoch 7/10 - Loss: 2.7587
Epoch 8/10 - Loss: 2.7289
Epoch 9/10 - Loss: 2.7141
Epoch 10/10 - Loss: 2.6858
Checkpoint saved to E:\dish dataset\checkpoints\simclr_epoch_10.pth
✅ Pretraining Complete.


In [14]:
pip install pandas

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.3 MB 6.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.3 MB 9.1 MB/s eta 0:00:02
   ---------------- ----------------------- 4.7/11.3 MB 8.9 MB/s eta 0:00:01
   ------------------------ --------------- 7.1/11.3 MB 9.7 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.3 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 10.3 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------

In [40]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

# Number of classes
num_classes = 20  # Your 20 fish classes

# Load pretrained ResNet18 backbone
model = resnet18(pretrained=True)

# Replace the final FC layer to match your classes
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Student/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


In [41]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Optional learning rate scheduler:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [42]:
from tqdm import tqdm

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(dataloader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [54]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from torch import nn, optim
from sklearn.model_selection import train_test_split

# === STEP 1: Paths ===
img_dir = r"E:\dish dataset\Fish Data Split\train"
csv_path = os.path.join(img_dir, "train_labels.csv")

# === STEP 2: Check Class Balance ===
df = pd.read_csv(csv_path)
print("📊 Class distribution:\n", df['label'].value_counts())

# === STEP 3: Transformations ===
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# === STEP 4: Custom Dataset ===
class FishDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# === STEP 5: Load Data and Split ===
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_df.to_csv("train_split.csv", index=False)
val_df.to_csv("val_split.csv", index=False)

train_dataset = FishDataset("train_split.csv", img_dir, transform=train_transform)
val_dataset = FishDataset("val_split.csv", img_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# === STEP 6: Define Model ===
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 20)  # 20 classes
)
model = model.cuda()

# === STEP 7: Training Setup ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# === STEP 8: Training and Validation Functions ===
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss, correct = 0.0, 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct / len(dataloader.dataset)
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss, correct = 0.0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct / len(dataloader.dataset)
    return epoch_loss, epoch_acc

# === STEP 9: Train Loop ===
device = 0  # GPU only
num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"|Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}|")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_fish_model.pth")
        print("✅ Best model saved.")



📊 Class distribution:
 label
14    1743
16    1564
18    1438
10    1264
0     1255
11    1221
7     1218
1     1174
13    1104
19    1015
15     995
17     890
5      713
12     673
6      640
8      582
4      391
9      387
2      302
3      296
Name: count, dtype: int64


c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Student\.conda\envs\torch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10]
|Train Loss: 0.5286 | Train Acc: 0.8640 | Val Loss: 0.0679 | Val Acc: 0.9828|
✅ Best model saved.
Epoch [2/10]
|Train Loss: 0.0702 | Train Acc: 0.9839 | Val Loss: 0.0250 | Val Acc: 0.9931|
✅ Best model saved.
Epoch [3/10]
|Train Loss: 0.0412 | Train Acc: 0.9906 | Val Loss: 0.0167 | Val Acc: 0.9958|
✅ Best model saved.
Epoch [4/10]
|Train Loss: 0.0279 | Train Acc: 0.9934 | Val Loss: 0.0133 | Val Acc: 0.9966|
✅ Best model saved.
Epoch [5/10]
|Train Loss: 0.0304 | Train Acc: 0.9920 | Val Loss: 0.0398 | Val Acc: 0.9886|
Epoch [6/10]
|Train Loss: 0.0092 | Train Acc: 0.9987 | Val Loss: 0.0037 | Val Acc: 0.9992|
✅ Best model saved.
Epoch [7/10]
|Train Loss: 0.0065 | Train Acc: 0.9991 | Val Loss: 0.0054 | Val Acc: 0.9984|
Epoch [8/10]
|Train Loss: 0.0054 | Train Acc: 0.9992 | Val Loss: 0.0061 | Val Acc: 0.9976|
Epoch [9/10]
|Train Loss: 0.0083 | Train Acc: 0.9983 | Val Loss: 0.0059 | Val Acc: 0.9981|
Epoch [10/10]
|Train Loss: 0.0095 | Train Acc: 0.9976 | Val Loss: 0.0103 | Val Ac

In [57]:
print(f"📈 Best Validation Accuracy: {best_val_acc * 100:.2f}%")


📈 Best Validation Accuracy: 99.92%
